In [13]:
import cv2
import os
import zipfile

def extract_frames(video_path, output_folder):
    video = cv2.VideoCapture(video_path)
    frame_count = 0
    
    with zipfile.ZipFile(os.path.join(output_folder, "frames.zip"), 'w') as zipf:
        while True:
            ret, frame = video.read()
            if not ret:
                break
            
            output_path = os.path.join(output_folder, f"frame_{frame_count}.png")
            cv2.imwrite(output_path, frame)
            zipf.write(output_path)
            os.remove(output_path)
            
            frame_count += 1
    video.release()

video_path = input("Enter the path to the video file: ")
output_folder = input("Enter the path to the output folder: ")
extract_frames(video_path, output_folder)
